In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np
import copy

# Settings for printing numpy arrays
np.set_printoptions(precision=4, edgeitems=50, suppress=True)
np.core.arrayprint._line_width = 250

# Second Partial Exam

**Ojeda Contreras Braulio Melquisedec**

**December 14th, 2022**

# Dual Simplex Method

Below is defined a function which comprises the Simplex Method helped by a Dual Problem which uses the table solution approach to solve Linear Programming Problems where we have to minimize or maximize an objective function under some restrictions.

In [3]:
def dual_simplex_tableau(tableau, variables, basic_indexes, non_basic_indexes, slack_indexes, itera = 1):
    
    basic_variables = [variables[i] for i in basic_indexes]
    non_basic_variables = [variables[i] for i in non_basic_indexes]
    
    # Printing our initial parameters for this iteration
    print("\n\n\033[1mIteration ", itera, "\033[0m")
    print("[\033[94m x_B \033[0m, \033[91m x_N \033[0m] = [ \033[94m"
          + ', '.join(basic_variables) + "\033[0m, \033[91m" + ', '.join(non_basic_variables) + "\033[0m ]\n")
    
    # Extracting elements from the tableau of the current iteration
    m = np.shape(tableau)[0] - 1
    n_plus_m = np.shape(tableau)[1] - 2
    ZX = np.ravel(tableau[0, 1:n_plus_m + 1])
    XBX = tableau[1:, 1:n_plus_m + 1]
    ZRHS = np.array([tableau[0, n_plus_m + 1]])
    XBRHS = (tableau[1:, n_plus_m + 1]).reshape(1, m)
    b_ = np.concatenate((np.ravel(XBRHS), np.zeros(n_plus_m - m)))
    
    print("\nEntry tableau")
    print(tableau)
    
    positive_b = True
    for b_i in np.ravel(XBRHS):
        if b_i < 0:
            positive_b = False
            break
    
    # Validating if we stop or not the optimization
    if positive_b:   # We stop
        
        # Printing the final results
        print("\n\n\033[1mOptimality reached\033[0m")
        print("\nThe optimal primal solution is")
        results_primal_ls = [str(round(i, 4)) for i in b_]
        print("\033[92m[ " + ', '.join(basic_variables + non_basic_variables) + " ] = [ " + ', '.join(results_primal_ls) + " ]\033[0m")
        perf_z = ZRHS[0]
        print("\nWith performance z =", perf_z)
        print("\nAnd the optimal dual solution is")
        results_dual_ls = np.ravel(-1 * tableau[0, [i + 1 for i in slack_indexes]]).tolist()
        results_dual_ls_f = [str(round(i, 4)) for i in results_dual_ls]
        print("\033[92m[ " + ', '.join(['w' + str(i + 1) for i in range(m)]) + " ] = [ " + ', '.join(results_dual_ls_f) + " ]\033[0m\n\n")
        
        indexes_variables = [int(var[1:]) for var in basic_variables + non_basic_variables]
        dict_primal_solution = dict(zip(indexes_variables, [b_i for b_i in b_]))
        sorted_dict_primal_solution = dict(sorted(dict_primal_solution.items(), key = lambda x:x[0]))
        primal_solution = np.array(list(sorted_dict_primal_solution.values()))
        
        dict_dual_solution = dict(zip([i for i in range(m)], results_dual_ls))
        sorted_dict_dual_solution = dict(sorted(dict_dual_solution.items(), key = lambda x:x[0]))
        dual_solution = np.array(list(sorted_dict_dual_solution.values()))
        
        #print(solution)
        return primal_solution, dual_solution
        
    else:   # We continue
        
        br = min(XBRHS)
        r = np.argmin(XBRHS)
        
        # Calculating and printing the y_ri's
        y_r = np.ravel(XBX[r, :])
        
        print("\nr =", r + 1, "-> row no.", r + 1, "of x_B part (", basic_variables[r], ")")
        print("y_r")
        print(y_r)
        
        # Analyzing if the Dual Problem is or not boundable by the condition y_r >= 0
        flag = True
        aux_counter = 0
        for y_i in y_r:
            if y_i >= 0:
                aux_counter += 1      
        if aux_counter == len(y_r):
            flag = False

        if flag:   # Boundable
            
            # Calculating and printing x_k by the minimum quotient (current BFS divided by the y_k > 0) with its index r
            quot_ls = list()
            index_k_ls = list()
            for i in range(n_plus_m):
                if y_r[i] < 0:
                    quot_ls.append(ZX[i] / y_r[i])
                    index_k_ls.append(i)
            quot_arr = np.array(quot_ls)
            index_k_arr = np.array(index_k_ls)
            k = index_k_arr[np.argmin(quot_arr)]
            min_quot = min(quot_arr)
            print("\nk =", k + 1, "-> column no.", k + 1, "of x part (", variables[k], ")")
            print("min_quotient")
            print(min_quot)
            
            # Pivoting
            pivot = y_r[k]
            print("\npivot =", pivot)
            
            tableau[r + 1, :] = tableau[r + 1, :] / pivot
            for i in range(m + 1):
                if i != r + 1:
                    tableau[i, :] = tableau[i, :] - tableau[i, k + 1] * tableau[r + 1, :]
            
            print("\nPivoted tableau")
            print(tableau)
            # Exchanging the indexes
            k_relative = 0
            for i in range(len(non_basic_indexes)):
                if k == non_basic_indexes[i]:
                    k_relative = i
                    break
            print("\n\033[94m", non_basic_variables[k_relative], "enters\033[0m and \033[91m", basic_variables[r], "leaves\033[0m the basis")
            aux_vars = basic_indexes[r]
            basic_indexes[r] = non_basic_indexes[k_relative]
            non_basic_indexes[k_relative] = aux_vars
            
            # Recursive call
            return dual_simplex_tableau(tableau, variables, basic_indexes, non_basic_indexes, slack_indexes, itera + 1)

        else:   # Not boundable
            
            print("\n\n\033[1mOptimization process stopped :(\033[0m")
            print("\nThe Dual Problem is not boundable, thus the primal feasible region is empty\n\n")
            
            return False, False
            

The Dual Simplex Method, as well as the Simplex Method implementations previously done, needs to preprocess our initial data in order to adjust it into a a particular tableau with the below entries:

|         | $z$ | $x_{B}$ | $x_{N}$                | $RHS$         |
|:---:|:---:|:---:|:---:|:---:|
| $z$     | $1$ | $0$     | $c_{b}B^{-1}N - c_{N}$ | $c_b \bar{b}$ |
| $X_{B}$ | $0$ | $I$     | $B^{-1}N$              | $\bar{b}$     |

That if we break it down we get the following equivalent tableau

|          | $z$      | $x_1$      | $x_2$      | $...$ | $x_n$      | $x_{n+1}$         | $...$ | $x_{n+m}$         | $RHS$         |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| $z$      | $1$      | $z_1-c_1$  | $z_2-c_2$  | $...$ | $z_n-c_n$  | $z_{n+1}-c_{n+1}$ | $...$ | $z_{n+m}-c_{n+m}$ | $c_b \bar{b}$ |
| $X_{B1}$ | $0$      | $Y_{1, 1}$ | $Y_{1, 2}$ | $...$ | $Y_{1, n}$ | $Y_{1, n+1}$      | $...$ | $Y_{1, n+m}$      | $\bar{b_1}$        |
| $X_{B2}$ | $0$      | $Y_{2, 1}$ | $Y_{2, 2}$ | $...$ | $Y_{2, n}$ | $Y_{2, n+1}$      | $...$ | $Y_{2, n+m}$      | $\bar{b_2}$          |
| $\vdots$ | $\vdots$ | $\vdots$   | $\vdots$   |       | $\vdots$   | $\vdots$          |       | $\vdots$          | $\vdots$      |
| $X_{Bm}$ | $0$      | $Y_{m, 1}$ | $Y_{m, 2}$ | $...$ | $Y_{m, n}$ | $Y_{m, n+1}$      | $...$ | $Y_{m, n+m}$      | $\bar{b_m}$          |

Taking this into account, the purpose of this method is to make the $\bar{b_i}$ less negative, and eventually turning them into positive values.

In [4]:
def construct_tableau(A, b, c, basic_indexes):
    m, n_plus_m = np.shape(A)
    B = A[:m, basic_indexes]
    c_b = c[basic_indexes]
    b_ = np.linalg.inv(B) @ b

    one_array = np.ones(1, dtype=int)
    zero_array_v = np.ravel(np.zeros(m, dtype=int)).reshape(1, m)
    ZX = np.ravel((c_b @ np.linalg.inv(B) @ A) - c)
    XBX = np.linalg.inv(B) @ A
    ZRHS = np.full((1), c_b @ np.ravel(b_))
    XBRHS = b_.reshape(1, m)

    tableau = np.vstack((np.concatenate((one_array, ZX, ZRHS), axis = 0).reshape(1, n_plus_m + 2),
                  np.concatenate((zero_array_v.T, XBX, XBRHS.T), axis = 1)))
    
    return tableau

In [5]:
import itertools

def basic_indexes_dual_feasible_solution(A, b, c, indexes, m):
    all_combinations = itertools.combinations(indexes, m)
    possible_basic_indexes = list()
    for basic_indexes in all_combinations:
        try:
            basic_indexes = list(basic_indexes)
            B = A[:m, basic_indexes]
            c_b = c[basic_indexes]
            ZX = np.ravel((c_b @ np.linalg.inv(B) @ A) - c)
            
            dual_feasible = True
            for zx in ZX:
                if zx > 0:
                    dual_feasible = False
                    break
            if dual_feasible:
                possible_basic_indexes.append(basic_indexes)  
        except:
            pass
    
    return possible_basic_indexes

In [6]:
def basic_variables_dual_feasible_solution(possible_basic_indexes, variables):
    possible_basic_variables = list()
    for psb in possible_basic_indexes:
        possible_basic_variables.append([variables[i_psb] for i_psb in psb])
    
    return possible_basic_variables

# Primal Dual Method

Below is defined a function which comprises the Primal Dual Method that is similar to the Dual Simplex Method but the main difference is that this method doesn't requiere a initial dual feasible solution to be basic.

In [7]:
def primal_dual(A, b, c, w, variables, itera = 1):
    
    # Printing our initial parameters for this iteration
    print("\n\n\033[1mIteration ", itera, "\033[0m")
    #print("[\033[94m x_B \033[0m, \033[91m x_N \033[0m] = [ \033[94m"
    #      + ', '.join(basic_variables) + "\033[0m, \033[91m" + ', '.join(non_basic_variables) + "\033[0m ]\n")
    
    # Extracting elements from the tableau of the current iteration
    m, n = np.shape(A)
    
    wA = np.ravel(w @ A)
    Q = list()
    for i in range(len(c)):
        if c[i] == wA[i]:
            Q.append(i)
    
    #print(wA)
    print("\nQ =", [q + 1 for q in Q])
    
    # Defining Restricted Primal Problem
    print("\n\033[94mRestricted Primal Problem\033[0m")
    no_variables = len(variables)
    artificial_variables = ['x' + str(i) for i in range(no_variables + 1, no_variables + m + 1)]
    variables_RPP = [variables[i] for i in range(no_variables) if i in Q] + artificial_variables
    print("\nVariables RPP")
    print(variables_RPP)
    
    identity_matrix = np.eye(m, dtype = int)
    extracted_A = A[:, Q]
    A_RPP = np.concatenate((extracted_A, identity_matrix), axis = 1)
    print("\nMatrix A RPP")
    print(A_RPP)
    
    c_RPP = np.concatenate((np.zeros(len(Q), dtype=int), np.ones(m, dtype=int)), axis = 0)
    print("\nVector c RPP")
    print(c_RPP)
    
    m_RPP, n_RPP = np.shape(A_RPP)
    
    basic_indexes = [ind for ind in range(n_RPP - m_RPP, n_RPP)]
    non_basic_indexes = [ind for ind in range(n_RPP - m_RPP)]
    tableau = construct_tableau(A_RPP, b, c_RPP, basic_indexes)
    
    solution_RPP = simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables_RPP), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes))
    print("\nSolution RPP")
    print(solution_RPP)
    
    x0 = c_RPP.T@solution_RPP
    print("\nPerformance RPP")
    print(x0)
    
    if x0 == 0:
        
        remaining_variables = [variables[i] for i in range(no_variables) if variables[i] not in variables_RPP]
        remaining_values = np.zeros(len(remaining_variables), dtype=int)
        
        indexes_variables = [int(var[1:]) for var in variables_RPP + remaining_variables]
        dict_primal_solution = dict(zip(indexes_variables, np.concatenate((solution_RPP, remaining_values), axis = 0).tolist()))
        sorted_dict_primal_solution = dict(sorted(dict_primal_solution.items(), key = lambda x:x[0]))
        primal_solution_ls = list(sorted_dict_primal_solution.values())[:-len(Q)]
        primal_variables_ls = list(sorted_dict_primal_solution.keys())[:-len(Q)]
        primal_variables_ls = ['x' + str(i) for i in primal_variables_ls]
        results_primal_ls = [str(round(i, 4)) for i in primal_solution_ls]
        dual_solution_ls = w.tolist()
        results_dual_ls = [str(round(i, 4)) for i in dual_solution_ls]
        
        print("\n\033[1mOptimality reached\033[0m")
        print("\nThe optimal primal solution is")
        print("\033[92m[ " + ', '.join(primal_variables_ls) + " ] = [ " + ', '.join(results_primal_ls) + " ]\033[0m")
        print("\nAnd the optimal dual solution is")
        print("\033[92m[ " + ', '.join(['w' + str(i + 1) for i in range(m)]) + " ] = [ " + ', '.join(results_dual_ls) + " ]\033[0m\n\n")
        
        return np.array(primal_solution_ls), w
        
    else:
        # Defining Dual of Restricted Primal Problem
        print("\n\033[94mDual of Restricted Primal Problem\033[0m")
        A_DRPP = np.concatenate((A_RPP.T, np.eye(len(variables_RPP), dtype = int)), axis = 1)
        b_DRPP = c_RPP
        c_DRPP = np.concatenate((-1 * b, np.zeros(len(variables_RPP), dtype = int)), axis = 0)
        artificial_variables_DRPP = ['x' + str(i) for i in range(no_variables + 1, no_variables + 2 * m + 1)]
        variables_DRPP = [variables[i] for i in range(no_variables) if i in Q] + artificial_variables_DRPP
        
        m_DRPP, n_DRPP = np.shape(A_DRPP)
        basic_indexes = [ind for ind in range(n_DRPP - m_DRPP, n_DRPP)]
        non_basic_indexes = [ind for ind in range(n_DRPP - m_DRPP)]
        tableau = construct_tableau(A_DRPP, b_DRPP, c_DRPP, basic_indexes)

        v = simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables_DRPP), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes))
        print(v)
        #v = simplex(copy.deepcopy(A_DRPP), copy.deepcopy(b_DRPP), copy.deepcopy(c_DRPP), copy.deepcopy(variables_DRPP))
        v = v[[index_w for index_w in range(m)]]
        
        flag_not_boundable = False
        if isinstance(v, bool) == True:
            flag_not_boundable = True
           
        
        if flag_not_boundable == False:
            print("\nv =", v)
            wa_minus_c = wA - c
            vA = np.ravel(v @ A)

            counter_flag = 0
            for i in range(no_variables):
                if vA[i] <= 0:
                    counter_flag += 1

            if counter_flag == no_variables:
                print("\n\033[1mOptimization process stopped :(\033[0m")
                print("\nThe Dual of RPP is unbounded and the primal is infeasible (vA <= 0)\n\n")

                return False, False

            else:
                quot_ls = [-1 * wa_minus_c[i] / vA[i] for i in range(no_variables) if vA[i] > 0]
                theta = min(quot_ls)
                print("\nTheta =", theta)

                w = w + theta * v
                print("\nw_updated =", w)

                return primal_dual(A, b, c, w, variables, itera + 1)
        else:
            print("\n\033[1mOptimization process stopped :(\033[0m")
            print("\nThe Dual of RPP is unbounded and the primal is infeasible\n\n")
            
            return False, False
            

To make possible the Primal Dual Method we are going to retake the functions of the Simplex Method in tableau format, which were made previously on other tasks.

In [8]:
def simplex_tableau(tableau, variables, basic_indexes, non_basic_indexes, itera = 1):
    
    basic_variables = [variables[i] for i in basic_indexes]
    non_basic_variables = [variables[i] for i in non_basic_indexes]
        
    # Printing our initial parameters for this iteration
    #print("\n\n\033[1mIteration ", itera, "\033[0m")
    #print("[\033[94m x_B \033[0m, \033[91m x_N \033[0m] = [ \033[94m"
    #      + ', '.join(basic_variables) + "\033[0m, \033[91m" + ', '.join(non_basic_variables) + "\033[0m ]\n")
    
    # Extracting elements from the tableau of the current iteration
    m = np.shape(tableau)[0] - 1
    n_plus_m = np.shape(tableau)[1] - 2
    ZXN = np.ravel(tableau[0, [i + 1 for i in non_basic_indexes]])
    XBXN = tableau[1:, [i + 1 for i in non_basic_indexes]]
    ZRHS = np.array([tableau[0, n_plus_m + 1]])
    XBRHS = (tableau[1:, n_plus_m + 1]).reshape(1, m)
    b_ = np.concatenate((np.ravel(XBRHS), np.zeros(n_plus_m - m)))
    
    #print("\nEntry tableau")
    #print(tableau)
    
    z_minus_c_max = max(ZXN)
    k = np.argmax(ZXN)
    
    # Validating if we stop or not the optimization
    if z_minus_c_max <= 0:   # We stop
        
        # Printing the final results
        #print("\n\n\033[1mOptimality reached\033[0m")
        #print("\nThe optimal BFS is")
        results_ls = [str(round(i, 4)) for i in b_]
        #print("\033[92m[ " + ', '.join(basic_variables + non_basic_variables) + " ] = [ " + ', '.join(results_ls) + " ]\033[0m")
        perf_z = ZRHS[0]
        #print("\nWith performance z =", perf_z, "\n\n")
        
        indexes_variables = [int(var[1:]) for var in basic_variables + non_basic_variables]
        dict_solution = dict(zip(indexes_variables, [b_i for b_i in b_]))
        sorted_dict_solution = dict(sorted(dict_solution.items(), key = lambda x:x[0]))
        solution = np.array(list(sorted_dict_solution.values()))
        #print(solution)
        return solution
        
    else:   # We continue
        
        # Calculating and printing the y_ki's
        y_k = np.ravel(XBXN[:, k])
        
        #print("\nk =", k + 1, "-> column no.", k + 1, "of x_N part (", non_basic_variables[k], ")")
        #print("y_k")
        #print(y_k)
        
        # Analyzing if the optimal BFS is or not boundable by the condition y_k > 0
        flag = True
        aux_counter = 0
        for y_i in y_k:
            if y_i > 0:
                aux_counter = aux_counter + 1      
        if aux_counter == 0:
            flag = False

        if flag:   # Boundable
            
            # Calculating and printing x_k by the minimum quotient (current BFS divided by the y_k > 0) with its index r
            quot_ls = list()
            index_r_ls = list()
            for i in range(m):
                if y_k[i] > 0:
                    quot_ls.append(b_[i] / y_k[i])
                    index_r_ls.append(i)
            quot_arr = np.array(quot_ls)
            index_r_arr = np.array(index_r_ls)
            r = index_r_arr[np.argmin(quot_arr)]
            x_Br = min(quot_arr)
            #print("\nr =", r + 1, "-> column no.", r + 1, "of x_B part (", basic_variables[r], ")")
            #print("x_Br")
            #print(x_Br)
            
            # Pivoting
            pivot = y_k[r]
            #print("\npivot =", pivot)
            
            tableau[r + 1, :] = tableau[r + 1, :] / pivot
            for i in range(m + 1):
                if i != r + 1:
                    tableau[i, :] = tableau[i, :] - tableau[i, non_basic_indexes[k] + 1] * tableau[r + 1, :]
            
            #print("\nPivoted tableau")
            #print(tableau)
            # Exchanging the indexes
            #print("\n\033[94m", non_basic_variables[k], "enters\033[0m and \033[91m", basic_variables[r], "leaves\033[0m the basis")
            aux_vars = basic_indexes[r]
            basic_indexes[r] = non_basic_indexes[k]
            non_basic_indexes[k] = aux_vars
            # Exchanging columns in tableau
            #aux_column = tableau[:, m + k + 1]
            #tableau[:, m + k + 1] =  tableau[:, 1 + r]
            #tableau[:, 1 + r] = aux_column
            
            # Recursive call
            return simplex_tableau(tableau, variables, basic_indexes, non_basic_indexes, itera + 1)

        else:   # Not boundable
            
            #print("\n\n\033[1mOptimization process stopped :(\033[0m")
            #print("\nThe optimal BFS is not boundable\n\n")
            
            return False
            

# Sensitivity in tableau format

Below are defined some cases when we modify an specific value from our initial Linear Programming Problem.

In [9]:
def sensitivity_costs_vector_c(tableau, original_costs, changed_costs, variables, basic_indexes, non_basic_indexes):
    
    basic_variables = [variables[i] for i in basic_indexes]
    non_basic_variables = [variables[i] for i in non_basic_indexes]
    
    # Extracting elements from the tableau of the current iteration
    m = np.shape(tableau)[0] - 1
    n_plus_m = np.shape(tableau)[1] - 2
    
    basic_change = False
    non_basic_change = False
    
    for i in range(n_plus_m):
        if original_costs[i] != changed_costs[i]:
            index_change = i
            ck = original_costs[i]
            ck_p = changed_costs[i]
            
            # Non basic change
            if index_change in non_basic_indexes:
                print("\n\033[1mSensitivity analysis: Changes on the costs vector\033[0m")
                print("Detected modification in \033[91mnon basic variable", variables[index_change], "\033[0m")
                non_basic_change = True
                z_minus_c_p = tableau[0, 1 + index_change] + (ck - ck_p)
                tableau[0, 1 + index_change] = z_minus_c_p
                if z_minus_c_p > 0:
                    print("\n[\033[94m x_B \033[0m, \033[91m x_N \033[0m] = [ \033[94m"
                          + ', '.join(basic_variables) + "\033[0m, \033[91m" + ', '.join(non_basic_variables) + "\033[0m ]")
                    print("\nThe problem needs the simplex method applied to it beginning with the resultant table")
                    print(tableau)
                    solution = simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes))
                    print("\nThe new solution is", solution)
                else:
                    print("\nThe changed solution remains optimal")
                    print(tableau)
            # Basic change
            else:
                print("\n\033[1mSensitivity analysis: Changes on the costs vector c\033[0m")
                print("Detected modification in \033[94mbasic variable", variables[index_change], "\033[0m")
                print("\n[\033[94m x_B \033[0m, \033[91m x_N \033[0m] = [ \033[94m"
                          + ', '.join(basic_variables) + "\033[0m, \033[91m" + ', '.join(non_basic_variables) + "\033[0m ]")
                basic_change = True
                k_relative = 0
                for i in range(len(basic_indexes)):
                    if index_change == basic_indexes[i]:
                        k_relative = i
                        break
                
                row_k_relative = np.ravel(tableau[k_relative + 1, :]) * (ck_p - ck)
                tableau[0, :] += row_k_relative
                tableau[0, index_change + 1] = 0
                
                index_positive = -1
                ZX = np.ravel(tableau[0, 1:n_plus_m + 1])
                for i in range(len(ZX)):
                    if ZX[i] > 0:
                        index_positive = i
                        break
                
                if index_positive in non_basic_indexes:
                    print("\nThe problem needs the simplex method applied to it beginning with the resultant table\n")
                    print(tableau)
                    solution = simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes))
                    print("\nThe new solution is", solution)
                else:
                    print("\nAlthough changes, the problem has the optimal primal table\n")
                    print(tableau)
            break
            

So, once we have already defined the previous function, we are able to apply it.

## Exercise 1 & 2:
Solve the next linear programming problem:

$$min \hspace{0.5cm} 2x_1 + 3x_2 + 5x_3 + 6x_4$$
$$under \hspace{0.5cm} x_1 + 2x_2 + 3x_3 + x_4 \geq 2$$
$$\hspace{1.4cm} -2x_1 + x_2 - x_3 + 3x_4 \leq -3$$
$$\hspace{1cm}x_1, x_2, x_3, x_4 \geq 0$$

First, we introduce our lack variables $x_5, x_6$ and then multiply by -1 the second constraint for mere convenience. So, the problem can be rewritten as

$$min \hspace{0.5cm} 2x_1 + 3x_2 + 5x_3 + 6x_4 + 0x_5 + 0x_6$$
$$under \hspace{0.5cm} x_1 + 2x_2 + 3x_3 + x_4 - x_5 + 0x_6 = 2$$
$$\hspace{1.4cm} 2x_1 - x_2 + x_3 - 3x_4 + 0x_5 - x_6 = 3$$
$$\hspace{1cm}x_1, x_2, x_3, x_4, x_5, x_6 \geq 0$$

And on the other hand, the associated dual problem is defined as

$$max \hspace{0.5cm} - 2w_1 - 3w_2$$
$$under \hspace{0.5cm} w_1 + 2w_2 \leq 2$$
$$\hspace{1.4cm} 2w_1 - w_2 \leq 3$$
$$\hspace{1.4cm} 3w_1 + w_2 \leq 5$$
$$\hspace{1.4cm} w_1 - 3w_2 \leq 6$$
$$\hspace{1.4cm} -w_1 + 0w_2 \leq 0$$
$$\hspace{1.4cm} 0w_1 - w_2 \leq 0$$
$$\hspace{1cm}w_1, w_2 \hspace{0.5cm}unrestricted$$

Then, initially considering $x_5, x_6$ as the basic variables and $x_1, x_2, x_3, x_4$ as the non basic ones, we get the following elements:

\begin{equation} A = [N | B] = 
\left[
\begin{array}{cccc | cc}
1 & 2 & 3 & 1 & -1 & 0\\
2 & -1 & 1 & -3 & 0 & -1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
2\\
3\\
\end{array}
\right]
\end{equation}


\begin{equation} c = [c_{N} | c_{B}] =
\left[
\begin{array}{cccc | cc}
2 & 3 & 5 & 6 & 0 & 0\\
\end{array}
\right]
\end{equation}

In [10]:
A = np.matrix([[1, 2, 3, 1, -1, 0], [2, -1, 1, -3, 0, -1]])
b = np.array([2, 3])
c = np.array([2, 3, 5, 6, 0, 0])
variables = ["x1", "x2", "x3", "x4", "x5", "x6"]

In [11]:
basic_indexes = [4, 5]
non_basic_indexes = [0, 1, 2, 3]
slack_indexes = [4, 5]

In [12]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [13]:
tableau

matrix([[ 1., -2., -3., -5., -6.,  0.,  0.,  0.],
        [ 0., -1., -2., -3., -1.,  1.,  0., -2.],
        [ 0., -2.,  1., -1.,  3.,  0.,  1., -3.]])

Note that since every $z_j - c_j \leq 0$ $\hspace{0.2cm}\forall j$, we can proceed to execute the simplex dual method.

In [14]:
dual_simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes), copy.deepcopy(slack_indexes))



Iteration  1 
[ x_B ,  x_N ] = [ x5, x6, x1, x2, x3, x4 ]


Entry tableau
[[ 1. -2. -3. -5. -6.  0.  0.  0.]
 [ 0. -1. -2. -3. -1.  1.  0. -2.]
 [ 0. -2.  1. -1.  3.  0.  1. -3.]]

r = 2 -> row no. 2 of x_B part ( x6 )
y_r
[-2.  1. -1.  3.  0.  1.]

k = 1 -> column no. 1 of x part ( x1 )
min_quotient
1.0

pivot = -2.0

Pivoted tableau
[[ 1.   0.  -4.  -4.  -9.   0.  -1.   3. ]
 [ 0.   0.  -2.5 -2.5 -2.5  1.  -0.5 -0.5]
 [-0.   1.  -0.5  0.5 -1.5 -0.  -0.5  1.5]]

 x1 enters and  x6 leaves the basis


Iteration  2 
[ x_B ,  x_N ] = [ x5, x1, x6, x2, x3, x4 ]


Entry tableau
[[ 1.   0.  -4.  -4.  -9.   0.  -1.   3. ]
 [ 0.   0.  -2.5 -2.5 -2.5  1.  -0.5 -0.5]
 [-0.   1.  -0.5  0.5 -1.5 -0.  -0.5  1.5]]

r = 1 -> row no. 1 of x_B part ( x5 )
y_r
[ 0.  -2.5 -2.5 -2.5  1.  -0.5]

k = 2 -> column no. 2 of x part ( x2 )
min_quotient
1.6

pivot = -2.5

Pivoted tableau
[[ 1.   0.   0.   0.  -5.  -1.6 -0.2  3.8]
 [-0.  -0.   1.   1.   1.  -0.4  0.2  0.2]
 [-0.   1.   0.   1.  -1.  -0.2 -0.4  1

(array([1.6, 0.2, 0. , 0. , 0. , 0. ]), array([1.6, 0.2]))

## Exercise 3:
Solve the next linear programming problem:

$$max \hspace{0.5cm} 10x_1 + 24x_2 + 20x_3 + 20x_4 + 25x_5$$
$$under \hspace{0.5cm} x_1 + x_2 + 2x_3 + 3x_4 + 5x_5 \leq 19$$
$$\hspace{1.4cm} 2x_1 + 4x_2 + 3x_3 + 2x_4 + x_5 \leq 57$$
$$\hspace{1cm}x_1, x_2, x_3, x_4, x_5 \geq 0$$

First, we introduce our lack variables $x_6, x_7$, then multiply by -1 the objective function in order to turn the maximization problem into a minimization one, and finally multiply by -1 the first and the second constraint for mere convenience. So, the problem can be rewritten as

$$min \hspace{0.5cm} - 10x_1 - 24x_2 - 20x_3 - 20x_4 - 25x_5 + 0x_6 + 0x_7$$
$$under \hspace{0.5cm} - x_1 - x_2 - 2x_3 - 3x_4 - 5x_5 - x_6 + 0x_7 = -19$$
$$\hspace{1.4cm} - 2x_1 - 4x_2 - 3x_3 - 2x_4 - x_5 + 0x_6 - x_7 = -57$$
$$\hspace{1cm}x_1, x_2, x_3, x_4, x_5, x_6, x_7 \geq 0$$

And on the other hand, the associated dual problem is defined as

$$max \hspace{0.5cm} - 19w_1 - 57w_2$$
$$under \hspace{0.5cm} - w_1 - 2w_2 \leq -10$$
$$\hspace{1.4cm} - w_1 - 4w_2 \leq -24$$
$$\hspace{1.4cm} - 2w_1 - 3w_2 \leq -20$$
$$\hspace{1.4cm} - 3w_1 - 2w_2 \leq -20$$
$$\hspace{1.4cm} - 5w_1 - w_2 \leq -25$$
$$\hspace{1.4cm} - w_1 + 0w_2 \leq 0$$
$$\hspace{1.4cm} 0w_1 - w_2 \leq 0$$
$$\hspace{1cm}w_1, w_2 \hspace{0.5cm}unrestricted$$

Then, to choose the basic variables by considering that a feasible solution for the dual problem is $w = [w_1, w_2] = [4, 5]$, we have to find such variables that comply $w = c_{B}B^{-1}$, i.e. $wB = c_{B}$.

Breaking down the above, we have that our matrix A is defined as

\begin{equation} A =
\left[
\begin{array}{ccccccc}
-1 & -1 & -2 & -3 & -5 & -1 & 0\\
-2 & -4 & -3 & -2 & -1 & 0 & -1\\
\end{array}
\right]
\end{equation}

So, to find basis B we retake the condition $wB = c_{B}$, which for this case looks like:

\begin{equation} wB =
[4,5]
\left[
\begin{array}{cc}
a_{1,j_1} & a_{1,j_2}\\
a_{2,j_1} & a_{2,j_2}\\
\end{array}
\right] = 
\left[
\begin{array}{cc}
c_{j_1} & c_{j_2}\\
\end{array}
\right]
\end{equation}

where $j_1, j_2$ are indexes of the variables that comply the equation.

After trying with different values we get that $j_1 = 2$ and $j_2 = 5$. Thus we are going to consider $x_2, x_5$ as the basic variables and $x_1, x_3, x_4, x_6, x_7$ as the non basic ones. So, we get the following elements:

\begin{equation} A = [N | B] = 
\left[
\begin{array}{ccccc | cc}
-1 & -2 & -3 & -1 & 0 & -1 & -5\\
-2 & -3 & -2 & 0 & -1 & -4 & -1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
-19\\
-57\\
\end{array}
\right]
\end{equation}


\begin{equation} c = [c_{N} | c_{B}] =
\left[
\begin{array}{ccccc | cc}
-10 & -20 & -20 & 0 & 0 & -24 & -25\\
\end{array}
\right]
\end{equation}

In [15]:
A = np.matrix([[-1, -1, -2, -3, -5, -1, 0], [-2, -4, -3, -2, -1, 0, -1]])
b = np.array([-19, -57])
c = np.array([-10, -24, -20, -20, -25, 0, 0])
variables = ["x1", "x2", "x3", "x4", "x5", "x6", "x7"]

In [16]:
basic_indexes = [1, 4]
non_basic_indexes = [0, 2, 3, 5, 6]
slack_indexes = [5, 6]

In [17]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [18]:
tableau

matrix([[   1.    ,   -4.    ,    0.    ,   -3.    ,   -2.    ,
            0.    ,   -4.    ,   -5.    , -361.    ],
        [   0.    ,    0.4737,    1.    ,    0.6842,    0.3684,
            0.    ,   -0.0526,    0.2632,   14.    ],
        [   0.    ,    0.1053,    0.    ,    0.2632,    0.5263,
            1.    ,    0.2105,   -0.0526,    1.    ]])

Note that since every $z_j - c_j \leq 0$ $\hspace{0.2cm}\forall j$, we can proceed to execute the simplex dual method.

In [19]:
dual_simplex_tableau(copy.deepcopy(tableau), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes), copy.deepcopy(slack_indexes))



Iteration  1 
[ x_B ,  x_N ] = [ x2, x5, x1, x3, x4, x6, x7 ]


Entry tableau
[[   1.       -4.        0.       -3.       -2.        0.       -4.
    -5.     -361.    ]
 [   0.        0.4737    1.        0.6842    0.3684    0.       -0.0526
     0.2632   14.    ]
 [   0.        0.1053    0.        0.2632    0.5263    1.        0.2105
    -0.0526    1.    ]]


Optimality reached

The optimal primal solution is
[ x2, x5, x1, x3, x4, x6, x7 ] = [ 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0 ]

With performance z = -361.0

And the optimal dual solution is
[ w1, w2 ] = [ 4.0, 5.0 ]




(array([ 0., 14.,  0.,  0.,  1.,  0.,  0.]), array([4., 5.]))

## Exercise 4:
Solve the next linear programming problem:

$$max \hspace{0.5cm} x_1 + 6x_2$$
$$under \hspace{0.5cm} x_1 + x_2 \geq 2$$
$$\hspace{1.4cm} x_1 + 3x_2 \leq 3$$
$$\hspace{1cm}x_1, x_2 \geq 0$$

First, we introduce our slack variables $x_3, x_4$ and then multiply by -1 the objective function in order to turn the maximization problem into a minimization one. So, the problem can be rewritten as

$$min \hspace{0.5cm} - x_1 - 6x_2 + 0x_3 + 0x_4$$
$$under \hspace{0.5cm} x_1 + x_2 - x_3 + 0x_4 = 2$$
$$\hspace{1.4cm} x_1 + 3x_2 + 0x_3 + x_4 = 3$$
$$\hspace{1cm}x_1, x_2, x_3, x_4 \geq 0$$

And on the other hand, the associated dual problem is defined as

$$max \hspace{0.5cm} 2w_1 + 3w_2$$
$$under \hspace{0.5cm} w_1 + w_2 \leq -1$$
$$\hspace{1.4cm} w_1 + 3w_2 \leq -6$$
$$\hspace{1.4cm} - w_1 + 0w_2 \leq 0$$
$$\hspace{1.4cm} 0w_1 + w_2 \leq 0$$
$$\hspace{1cm}w_1, w_2 \hspace{0.5cm}unrestricted$$


Then, we get the following elements:

\begin{equation} A = 
\left[
\begin{array}{cccc}
1 & 1 & -1 & 0\\
1 & 3 & 0 & 1\\
\end{array}
\right]
\end{equation}

\begin{equation} b = 
\left[
\begin{array}{r}
2\\
3\\
\end{array}
\right]
\end{equation}


\begin{equation} c =
\left[
\begin{array}{cccc}
-1 & -6 & 0 & 0\\
\end{array}
\right]
\end{equation}

In [20]:
A = np.matrix([[1, 1, -1, 0], [1, 3, 0, 1]])
b = np.array([2, 3])
c = np.array([-1, -6, 0, 0])
variables = ["x1", "x2", "x3", "x4"]

Besides that, we propose $w = [1.5, -2.5]$ as our initial dual feasible solution that will enable us to start the Primal Dual Method.

In [21]:
w = np.array([1.5, -2.5])

In [22]:
primal_dual(copy.deepcopy(A), copy.deepcopy(b), copy.deepcopy(c), copy.deepcopy(w), copy.deepcopy(variables))



Iteration  1 

Q = [1, 2]

Restricted Primal Problem

Variables RPP
['x1', 'x2', 'x5', 'x6']

Matrix A RPP
[[1 1 1 0]
 [1 3 0 1]]

Vector c RPP
[0 0 1 1]

Solution RPP
[1.5 0.5 0.  0. ]

Performance RPP
0.0

Optimality reached

The optimal primal solution is
[ x1, x2, x3, x4 ] = [ 1.5, 0.5, 0.0, 0.0 ]

And the optimal dual solution is
[ w1, w2 ] = [ 1.5, -2.5 ]




(array([1.5, 0.5, 0. , 0. ]), array([ 1.5, -2.5]))

## Exercise 5

Consider the following problem and its optimal tableau:

$$max \hspace{0.5cm} 2x_1 + x_2 - x_3$$
$$under \hspace{0.5cm} x_1 + 2x_2 + x_3 \leq 8$$
$$\hspace{1.4cm} -x_1 + x_2 - 2x_3 \leq 4$$
$$\hspace{1cm}x_1, x_2, x_3 \geq 0$$

|     | $z$ | $x_1$ | $x_2$ | $x_3$ | $x_4$ | $x_5$ | $RHS$ |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| $z$ | $1$ | $0$ | $3$ | $3$ | $2$ | $0$ | $16$ |
| $X_{1}$ | $0$ | $1$ | $2$ | $1$ | $1$ | $0$ | $8$ |
| $X_{5}$ | $0$ | $0$ | $3$ | $-1$ | $1$ | $1$ | $12$ |


a) Using sensitivity analysis find the new optimal solution if the costs vector c changes as following:

\begin{equation} c =
\left[
\begin{array}{ccccc}
-2 & -1 & 1 & 0 & 0\\
\end{array}
\right]
\end{equation}

\begin{equation} c' =
\left[
\begin{array}{ccccc}
-2 & -5 & 1 & 0 & 0\\
\end{array}
\right]
\end{equation}

First, for mere convenience we rewrite the problem as

$$min \hspace{0.5cm} -2x_1 - x_2 + x_3 + 0x_4 + 0x_5$$
$$under \hspace{0.5cm} - x_1 - 2x_2 - x_3 - x_4 + 0 x_5 = -8$$
$$\hspace{1.4cm} x_1 - x_2 + 2x_3 + 0x_4 - x_5 = -4$$
$$\hspace{1cm}x_1, x_2, x_3, x_4, x_5 \geq 0$$

In [23]:
A = np.array([[1, 2, 1, 1, 0],
             [-1, 1, -2, 0, 1]])
b = np.array([8, 4])
c = np.array([-2, -1, 1, 0, 0])

In [24]:
variables = ["x1", "x2", "x3", "x4", "x5"]
basic_indexes = [0, 4]
non_basic_indexes = [1, 2, 3]
slack_indexes = [3, 4]

In [25]:
tableau = construct_tableau(A, b, c, basic_indexes)

In [26]:
tableau

array([[  1.,   0.,  -3.,  -3.,  -2.,   0., -16.],
       [  0.,   1.,   2.,   1.,   1.,   0.,   8.],
       [  0.,   0.,   3.,  -1.,   1.,   1.,  12.]])

So the equivalent optimal table is

|     | $z$ | $x_1$ | $x_2$ | $x_3$ | $x_4$ | $x_5$ | $RHS$ |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| $z$ | $1$ | $0$ | $-3$ | $-3$ | $-2$ | $0$ | $-16$ |
| $X_{1}$ | $0$ | $1$ | $2$ | $1$ | $1$ | $0$ | $8$ |
| $X_{5}$ | $0$ | $0$ | $3$ | $-1$ | $1$ | $1$ | $12$ |

and with this information we can proceed with the sensitivity analysis

In [27]:
optimal_tableau = np.array([[1., 0., -3., -3., -2., 0., -16.],
                            [0., 1., 2., 1., 1., 0., 8.],
                            [0., 0., 3., -1., 1., 1., 12.]])

In [28]:
optimal_tableau

array([[  1.,   0.,  -3.,  -3.,  -2.,   0., -16.],
       [  0.,   1.,   2.,   1.,   1.,   0.,   8.],
       [  0.,   0.,   3.,  -1.,   1.,   1.,  12.]])

In [29]:
original_costs = np.array([-2, -1, 1, 0, 0])
changed_costs = np.array([-2, -5, 1, 0, 0])

In [30]:
sensitivity_costs_vector_c(copy.deepcopy(optimal_tableau), copy.deepcopy(original_costs), copy.deepcopy(changed_costs), copy.deepcopy(variables), copy.deepcopy(basic_indexes), copy.deepcopy(non_basic_indexes))


Sensitivity analysis: Changes on the costs vector
Detected modification in non basic variable x2 

[ x_B ,  x_N ] = [ x1, x5, x2, x3, x4 ]

The problem needs the simplex method applied to it beginning with the resultant table
[[  1.   0.   1.  -3.  -2.   0. -16.]
 [  0.   1.   2.   1.   1.   0.   8.]
 [  0.   0.   3.  -1.   1.   1.  12.]]

The new solution is [0. 4. 0. 0. 0.]
